Modified from https://lectures.quantecon.org/jl/mccall_model.html

In [51]:
function solve_mccall_model2(mcm; tol=1e-5, max_iter=2000, U_iv = 1.0, V_iv = ones(length(mcm.w)))
    α, β, σ, c, γ, w = mcm.α, mcm.β, mcm.σ, mcm.c, mcm.γ, mcm.w

    #Calculate the utility at the wages and unemployment
    u = mcm.u(w, σ)
    u_c = mcm.u(c, σ) #Utility of unemployment benefits
    
    #Initial values
    V = V_iv
    V_new = V_iv
    U = V_iv
    error = Inf

    #Value function iteration
    iter = 1
    while error > tol && iter < max_iter
        V_new = u + β * ((1 - α) * V + α * U)
        U_new = u_c + β * (1 - γ) * U + β * γ * dot(max.(U, V), mcm.p)

        error = norm([V_new - V; U_new - U], Inf)
        V = V_new
        U = U_new
        iter += 1
    end

    return V, U
end

solve_mccall_model2 (generic function with 1 method)

In [50]:
using Plots
gr()

#Utility function
u(c, σ) = (c.^(1 - σ) - 1) ./ (1 - σ)

#Wage values
n = 10
w = linspace(10, 20, n)

#The distribution to use for probabilities.
dist = BetaBinomial(n - 1, 600, 400)
p = pdf.(dist, support(dist))

mcm = @NT( #Can drop @NT with Julia v0.7/v1.0
    u = u,
    α = 0.2,
    β = 0.98,
    γ = 0.7,
    c = 6.0,
    σ = 2.0,
    w = w,
    p = p)

V, U = solve_mccall_model2(mcm)
U_vec = U .* ones(length(mcm.w))

plot(mcm.w, 
     [V U_vec],
     lw = 2, 
     α = 0.7, 
     label=["V" "U"])

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 17.5 
 
 
 20.0 
 
 
 45.55 
 
 
 45.60 
 
 
 45.65 
 
 
 45.70 
 
 
 45.75 
 
 
 
 
 
 
 
 V 
 
 
 
 U

In [ ]:
"""
Computes the reservation wage of an instance of the McCall model
by finding the smallest w such that V(w) > U.

If V(w) > U for all w, then the reservation wage w_bar is set to
the lowest wage in mcm.w_vec.

If v(w) < U for all w, then w_bar is set to np.inf.

Parameters
----------
mcm : an instance of McCallModel
return_values : bool (optional, default=false)
    Return the value functions as well

Returns
-------
w_bar : scalar
    The reservation wage

"""
function compute_reservation_wage(mcm::McCallModel; return_values::Bool=false)

    V, U = solve_mccall_model(mcm)
    w_idx = searchsortedfirst(V - U, 0)

    if w_idx == length(V)
        w_bar = Inf
    else
        w_bar = mcm.w_vec[w_idx]
    end

    if return_values == false
        return w_bar
    else
        return w_bar, V, U
    end

end

In [ ]:
grid_size = 25
γ_vals = linspace(0.05, 0.95, grid_size)

In [ ]:
using Plots
gr()


grid_size = 25  
c_vals = linspace(2, 12, grid_size)  
w_bar_vals = similar(c_vals)

mcm = McCallModel()

for (i, c) in enumerate(c_vals)
    mcm.c = c
    w_bar = compute_reservation_wage(mcm)
    w_bar_vals[i] = w_bar
end

plot(c_vals, 
     w_bar_vals, 
     lw=2, 
     α=0.7, 
     xlabel="unemployment compensation",
     ylabel="reservation wage",
     label="w bar as a function of c")

In [ ]:
grid_size = 25  
γ_vals = linspace(0.05, 0.95, grid_size)  
w_bar_vals = similar(γ_vals)

mcm = McCallModel()

for (i, γ) in enumerate(γ_vals)
    mcm.γ = γ
    w_bar = compute_reservation_wage(mcm)
    w_bar_vals[i] = w_bar
end

plot(γ_vals, 
     w_bar_vals, 
     lw=2, 
     α=0.7, 
     xlabel="job offer rate",
     ylabel="reservation wage",
     label="w bar as a function of \\gamma")